In [ ]:
# download and preprocess data
# put the final results to the data directory
# load it to server and run as in task 3.3-4

## SODA

In [1]:
from datasets import load_dataset

dataset = load_dataset("allenai/soda", split='test')

Using custom data configuration allenai--soda-354e990899ae2f4a


Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Dataset parquet downloaded and prepared to /Users/vladimirknazevskij/.cache/huggingface/datasets/allenai___parquet/allenai--soda-354e990899ae2f4a/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec. Subsequent calls will reuse this data.


In [4]:
dialogues = dataset['dialogue']

In [18]:
import numpy as np

chosen_idx = np.random.choice(len(dialogues), size=10 ** 4, replace=False)

In [23]:
chosen_dialogue = []
for idx in chosen_idx:
    dialogue = dialogues[idx]
    start = np.random.choice(len(dialogue) - 1)
    end = start + 1
    chosen_dialogue.append(f'\"{dialogue[start]}\"\n\"{dialogue[end]}\"')
    

In [26]:
import pandas as pd

df = pd.DataFrame(chosen_dialogue, columns=['text'])

In [28]:
df.to_csv('data/soda.csv', index=False)

In [29]:
! pwd

/Users/vladimirknazevskij/Jupyter/Humor/unlabeled


## MELD

In [1]:
! wget http://web.eecs.umich.edu/~mihalcea/downloads/MELD.Raw.tar.gz

URL transformed to HTTPS due to an HSTS policy
--2024-05-13 23:35:30--  https://web.eecs.umich.edu/~mihalcea/downloads/MELD.Raw.tar.gz
Resolving web.eecs.umich.edu (web.eecs.umich.edu)... 141.212.113.214
Connecting to web.eecs.umich.edu (web.eecs.umich.edu)|141.212.113.214|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10878146150 (10G) [application/x-gzip]
Saving to: ‘MELD.Raw.tar.gz’

MELD.Raw.tar.gz     100%[===================>]  10,13G  9,63MB/s    in 31m 27s 

2024-05-14 00:06:59 (5,50 MB/s) - ‘MELD.Raw.tar.gz’ saved [10878146150/10878146150]



In [3]:
! tar -xvzf MELD.Raw.tar.gz

x MELD.Raw/
x MELD.Raw/train.tar.gz
x MELD.Raw/dev.tar.gz
x MELD.Raw/test_sent_emo.csv
x MELD.Raw/._train_splits
x MELD.Raw/dev_sent_emo.csv
x MELD.Raw/README.txt
x MELD.Raw/test.tar.gz


In [5]:
import pandas as pd

train = pd.read_csv('MELD.Raw/train_sent_emo.csv')
valid = pd.read_csv('MELD.Raw/train_sent_emo.csv')
test = pd.read_csv('MELD.Raw/train_sent_emo.csv')

final = pd.concat((train, valid, test), axis=0, ignore_index=True)

In [6]:
final

,Sr No.,Utterance,Speaker,Emotion,Sentiment,Dialogue_ID,Utterance_ID,Season,Episode,StartTime,EndTime
0,1,also I was the point person on my companys tr...,Chandler,neutral,neutral,0,0,8,21,"00:16:16,059","00:16:21,731"
1,2,You mustve had your hands full.,The Interviewer,neutral,neutral,0,1,8,21,"00:16:21,940","00:16:23,442"
2,3,That I did. That I did.,Chandler,neutral,neutral,0,2,8,21,"00:16:23,442","00:16:26,389"
3,4,So lets talk a little bit about your duties.,The Interviewer,neutral,neutral,0,3,8,21,"00:16:26,820","00:16:29,572"
4,5,My duties? All right.,Chandler,surprise,positive,0,4,8,21,"00:16:34,452","00:16:40,917"
...,...,...,...,...,...,...,...,...,...,...,...
29962,10474,You or me?,Chandler,neutral,neutral,1038,13,2,3,"00:00:48,173","00:00:50,799"
29963,10475,"I got it. Uh, Joey, women don't have Adam's ap...",Ross,neutral,neutral,1038,14,2,3,"00:00:51,009","00:00:53,594"
29964,10476,"You guys are messing with me, right?",Joey,surprise,positive,1038,15,2,3,"00:01:00,518","00:01:03,520"
29965,10477,Yeah.,All,neutral,neutral,1038,16,2,3,"00:01:05,398","00:01:07,274"


In [1]:
emotions = ['neutral', 'joy', 'surprise', 'anger', 'sadness', 'disgust', 'fear']

In [13]:
prev = None

emotion_pairs = []
texts = []

for i in final.iterrows():
    idx, item = i
    if prev is not None:
        if prev['Dialogue_ID'] == item['Dialogue_ID']:
            texts.append(f'\"{prev["Utterance"]}\"\n\"{item["Utterance"]}\"')
            emotion_pairs.append((prev['Emotion'], item['Emotion']))
        
    prev = item

In [15]:
df = pd.DataFrame({'text': texts, 'emotion_pair': emotion_pairs})

In [27]:
from tqdm import tqdm

for e in emotions[1:]:
    print(e)
    texts = []
    for i in tqdm(range(len(df))):
        if e in df.iloc[i].emotion_pair:
            texts.append(df.iloc[i].text)
    df_ = pd.DataFrame({'text': texts})
    df_.to_csv(f'data/MELD_{e}.csv', index=False)

joy


100%|██████████████████████████████████| 26853/26853 [00:00<00:00, 28560.09it/s]


surprise


100%|██████████████████████████████████| 26853/26853 [00:00<00:00, 31236.98it/s]


anger


...

100%|██████████████████████████████████| 26853/26853 [00:00<00:00, 31801.92it/s]


disgust


100%|██████████████████████████████████| 26853/26853 [00:00<00:00, 35496.50it/s]


fear


100%|██████████████████████████████████| 26853/26853 [00:00<00:00, 35941.25it/s]


In [2]:
for i in emotions:
    print(f'MELD_{i}')

MELD_neutral
MELD_joy
MELD_surprise
MELD_anger
MELD_sadness
MELD_disgust
MELD_fear
